# Figures 8 and 9

HAMP observation and retrieval for a northbound flight segment above a stratocumulus field on 12 April 2022.

HAMP observation and retrieval for a southbound flight segment during the warm air intrusion on 14 March 2022.

In [ ]:
from string import ascii_lowercase

import cmcrameri.cm as cmc
import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
import numpy as np
import pandas as pd
from lizard.mpltools import style
from lizard.readers.band_pass import read_band_pass
from lizard.readers.mira import read_mira
from lizard.readers.wales import read_wales
from lizard.readers.worldview import read_worldview
from lizard.writers.figure_to_file import write_figure

from si_clouds.helpers.sigmoid import sigmoid
from si_clouds.io.readers.ancillary import read_ancillary_data
from si_clouds.io.readers.oem_result import read_oem_result_concat
from si_clouds.io.readers.sensitivity import read_sensitivity_params
from si_clouds.io.readers.specmacs import read_specmacs
from si_clouds.io.readers.velox import read_velox

In [ ]:
figure = "fig08"

versions = {"fig08": "pub_r2_case_6_v1", "fig09": "pub_r2_case_1_v1"}
flight_ids = {
    "fig08": "HALO-AC3_HALO_RF18",
    "fig09": "HALO-AC3_HALO_RF04",
}

version = versions[figure]
flight_id = flight_ids[figure]

In [ ]:
ds_bp = read_band_pass("HAMP")
ds_anc = read_ancillary_data()
popt = read_sensitivity_params()

In [ ]:
test_id = ""
write = False
ds_a, ds_op, _, _ = read_oem_result_concat(
    version, test_id, write=write
)

# remove times where the retrieval was not valid (ze or 2m temperature)
ds_a = ds_a.sel(time=ds_anc.ix_retrieval_valid.sel(time=ds_a.time))
ds_op = ds_op.sel(time=ds_anc.ix_retrieval_valid.sel(time=ds_op.time))

In [ ]:
ds_spm = read_specmacs(flight_id)
ds_wales = read_wales(flight_id, product="bsrgl")
ww_data, ww_extent = read_worldview(flight_id)
ds_vlx = read_velox(flight_id)
ds_mira = read_mira(flight_id)

In [ ]:
t0, t1 = ds_a.time.min().values, ds_a.time.max().values

# h_max, locator_maj, locator_min
height_dct = {
    "pub_r2_case_1_v1": (7, 5, 1),
    "pub_r2_case_6_v1": (2, 1, 0.5),
}

# vmin and vmax
nir_dct = {
    "pub_r2_case_1_v1": (0, 12),
    "pub_r2_case_6_v1": (0, 50),
}

cwp_ylim = {
    "pub_r2_case_1_v1": (0, 400),
    "pub_r2_case_6_v1": (0, 200),
}

colors_dct = {
    "pub_r2_case_1_v1": ["k"] * 16,
    "pub_r2_case_6_v1": ["k", "k", "lightgray"] + ["k"] * 13,
}

In [ ]:
print("Convergence fraction", ds_a.conv.sel(time=slice(t0, t1)).mean("time").item())

In [ ]:
# compute distance as a function of time between start and end as a function of ground speed
da_vel = ds_anc.vel.sel(time=slice(t0, t1))

assert da_vel.time.diff("time").max() < np.timedelta64(60, "s")

# resample to 1s by linear interpolation
time_1s = pd.date_range(t0, t1, freq="1s")
da_vel = da_vel.interp(time=time_1s)

assert np.sum(np.isnan(da_vel)) == 0
assert da_vel.min() > 150
assert da_vel.max() < 400
assert da_vel.time.diff("time").max()
assert da_vel.time.diff("time").min() == np.timedelta64(1, "s")
assert da_vel.time.diff("time").max() == np.timedelta64(1, "s")
da_dist = (da_vel.cumsum() - da_vel.isel(time=0)).rename("distance") * 1e-3

# provide the same on the specmacs times by interpolating distance linearly
da_dist_spm = da_dist.interp(time=ds_spm.time)

In [ ]:
fig, axes = plt.subplots(
    8, 2, figsize=(7, 8), sharex=True, layout="constrained"
)

for i, ax in enumerate(fig.axes):
    j = i % 2  # column index
    k = i // 2  # row index (letter)
    ax.annotate(
        f"({ascii_lowercase[k]}{j+1})",
        xy=(0.995, 0.99),
        xycoords="axes fraction",
        ha="right",
        va="top",
        color=colors_dct[version][i],
    )

ax_nir = axes[0, 0]
ax_tir = axes[1, 0]
ax_radar = axes[0, 1]
ax_wales = axes[1, 1]
axes_y = axes[2:, 0]
axes_x = axes[2:, 1]

ax_cwp = axes_x[0]
ax_tas = axes_x[1]
ax_tsi = axes_x[2]
ax_wscl = axes_x[3]
ax_dhcl = axes_x[4]
ax_wsd = axes_x[5]

axes_y[0].annotate(
    "Observation space",
    xy=(0.5, 1),
    xycoords="axes fraction",
    ha="center",
    va="bottom",
)
axes_x[0].annotate(
    "State space",
    xy=(0.5, 1),
    xycoords="axes fraction",
    ha="center",
    va="bottom",
)

kwds = dict(s=5, lw=0, marker=".")

# specmacs 1024 nm radiance
im_nir = ax_nir.pcolormesh(
    da_dist_spm.sel(time=ds_spm.time.sel(time=slice(t0, t1))),
    ds_spm.x,
    ds_spm.radiance.sel(time=slice(t0, t1)),
    cmap="Greys_r",
    zorder=0,
    vmin=nir_dct[version][0],
    vmax=nir_dct[version][1],
)
ax_nir.set_ylim(-15, 15)
ax_nir.set_yticks(np.arange(-10, 10, 5), minor=True)
ax_nir.set_ylabel("Angle [°]")

# velox
factor = np.ones(len(ds_vlx.vza))
factor[ds_vlx.vza.argmin().item() :] = -1
im_tir = ax_tir.pcolormesh(
    da_dist.sel(time=ds_vlx.time.sel(time=slice(t0, t1))),
    ds_vlx.vza * factor,
    ds_vlx.BT_2D.sel(time=slice(t0, t1)).T,
    cmap=cmc.lipari,
    zorder=0,
    vmin=235,
    vmax=275,
)
ax_tir.set_ylim(-15, 15)
ax_tir.set_yticks(np.arange(-10, 10, 5), minor=True)
ax_tir.set_ylabel("Angle [°]")

# colorbars
cb_ax_left = ax_nir.inset_axes([0, 1.05, 0.47, 0.1])
cb_ax_right = ax_nir.inset_axes([0.53, 1.05, 0.47, 0.1])
cb_left = fig.colorbar(
    im_nir,
    cax=cb_ax_left,
    label="NIR rad. (1 $\mu$m)\n[mW m$^{-2}$ nm$^{-1}$ sr$^{-1}$]",
    orientation="horizontal",
)
cb_right = fig.colorbar(
    im_tir,
    cax=cb_ax_right,
    label="TIR $T_b$ (10.7 $\mu$m)\n[K]",
    orientation="horizontal",
    ticks=[240, 255, 270],
)

for cb in [cb_left, cb_right]:
    cb.ax.xaxis.set_label_position("top")
    cb.ax.xaxis.set_ticks_position("top")

# radar
im_radar = ax_radar.pcolormesh(
    da_dist.sel(time=ds_mira["time"].sel(time=slice(t0, t1))),
    ds_mira["height"] * 1e-3,
    ds_mira["dBZg"].sel(time=slice(t0, t1)).T,
    cmap=cmc.batlow,
    shading="nearest",
    vmin=-35,
    vmax=20,
)
ax_radar.set_ylabel("Hgt. [km]")
ax_radar.set_ylim(0, height_dct[version][0])
ax_radar.yaxis.set_major_locator(
    mticker.MultipleLocator(height_dct[version][1])
)
ax_radar.yaxis.set_minor_locator(
    mticker.MultipleLocator(height_dct[version][2])
)

# wales
cmap = cmc.davos_r.copy()
cmap.set_bad("gray")
norm = mcolors.LogNorm(1, 200)
im_wales = ax_wales.pcolormesh(
    da_dist.sel(time=ds_wales["time"].sel(time=slice(t0, t1))),
    ds_wales["altitude"] * 1e-3,
    ds_wales["backscatter_ratio"]
    .where(ds_wales["flags"] == 0)
    .sel(time=slice(t0, t1))
    .T,
    cmap=cmap,
    shading="nearest",
    norm=norm,
    zorder=0,
)
ax_wales.set_ylabel("Hgt. [km]")
ax_wales.set_ylim(0, height_dct[version][0])
ax_wales.yaxis.set_major_locator(
    mticker.MultipleLocator(height_dct[version][1])
)
ax_wales.yaxis.set_minor_locator(
    mticker.MultipleLocator(height_dct[version][2])
)

# colorbars
cb_ax_left = ax_radar.inset_axes([0, 1.05, 0.47, 0.1])
cb_ax_right = ax_radar.inset_axes([0.53, 1.05, 0.47, 0.1])
cb_left = fig.colorbar(
    im_radar,
    cax=cb_ax_left,
    label="$Z_e$\n[dBZ]",
    orientation="horizontal",
    ticks=[-30, -15, 0, 15],
)
cb_right = fig.colorbar(
    im_wales,
    cax=cb_ax_right,
    label="Backscatter ratio\n(532 nm)",
    orientation="horizontal",
    ticks=[1, 100],
)

for cb in [cb_left, cb_right]:
    cb.ax.xaxis.set_label_position("top")
    cb.ax.xaxis.set_ticks_position("top")

# tb space
for i, channel in enumerate(ds_a.channel.values):
    ax = axes_y[i]

    # plot shadings
    y_err_obs = np.sqrt(ds_a.unc_y.sel(channel1=channel, channel2=channel))
    y_err_a = np.sqrt(
        ds_a.unc_meas_eff.sel(update=0, channel1=channel, channel2=channel)
    )
    y_err_op = np.sqrt(
        ds_op.unc_meas_eff.sel(
            update=ds_op.conv_i, channel1=channel, channel2=channel
        )
    )
    ax.fill_between(
        da_dist.sel(time=ds_a.time),
        ds_a.y_obs.sel(channel=channel) - y_err_obs,
        ds_a.y_obs.sel(channel=channel) + y_err_obs,
        color="k",
        linewidths=0,
        alpha=0.25,
    )
    ax.fill_between(
        da_dist.sel(time=ds_a.time),
        ds_a.y_sim.sel(channel=channel) - y_err_a,
        ds_a.y_sim.sel(channel=channel) + y_err_a,
        color="gray",
        linewidths=0,
        alpha=0.25,
    )
    ax.fill_between(
        da_dist.sel(time=ds_op.time),
        ds_op.y_sim.sel(channel=channel) - y_err_op,
        ds_op.y_sim.sel(channel=channel) + y_err_op,
        color="coral",
        linewidths=0,
        alpha=0.25,
    )

    ax.scatter(
        da_dist.sel(time=ds_a.time),
        ds_a.y_obs.sel(channel=channel),
        color="k",
        label="Obs.",
        **kwds,
    )
    ax.scatter(
        da_dist.sel(time=ds_a.time),
        ds_a.y_sim.sel(channel=channel),
        color="gray",
        label="A priori",
        **kwds,
    )
    ax.scatter(
        da_dist.sel(time=ds_op.time),
        ds_op.y_sim.sel(channel=channel),
        color="coral",
        label="Optimal",
        **kwds,
    )
    ax.yaxis.set_major_locator(mticker.MultipleLocator(30))
    ax.yaxis.set_minor_locator(mticker.MultipleLocator(10))
    ax.set_ylabel("$T_b$ [K]")
    ax.annotate(
        ds_bp.label.sel(channel=channel).values,
        xy=(0.01, 0.01),
        xycoords="axes fraction",
        ha="left",
        va="bottom",
    )

# state space
x_vars = [
    "cwp",
    "t_as",
    "t_si",
    "wind_slab_corr_length",
    "depth_hoar_corr_length",
    "wind_slab_thickness",
]
labels = [
    r"CWP [g m$^{-2}$]",
    r"$T_{as}$ [K]",
    r"$T_{si}$ [K]",
    r"$\xi_{WS}$ [mm]",
    r"$\xi_{DH}$ [mm]",
    r"$h_{WS}$ [cm]",
]
loc_maj = [200, 10, 10, 0.1, 0.2, 10]
loc_min = [50, 2, 2, 0.02, 0.05, 2]
factor = [1e3, 1, 1, 1, 1, 1e2]
for i, x_var in enumerate(x_vars):
    ax = axes_x[i]
    ax.fill_between(
        da_dist.sel(time=ds_a.time),
        (ds_a[x_var] - ds_a[x_var + "_std"]) * factor[i],
        (ds_a[x_var] + ds_a[x_var + "_std"]) * factor[i],
        color="gray",
        linewidths=0,
        alpha=0.25,
    )
    ax.fill_between(
        da_dist.sel(time=ds_op.time),
        (ds_op[x_var] - ds_op[x_var + "_std"]) * factor[i],
        (ds_op[x_var] + ds_op[x_var + "_std"]) * factor[i],
        color="coral",
        linewidths=0,
        alpha=0.25,
    )
    ax.scatter(
        da_dist.sel(time=ds_a.time),
        ds_a[x_var] * factor[i],
        color="gray",
        **kwds,
    )
    ax.scatter(
        da_dist.sel(time=ds_op.time),
        ds_op[x_var] * factor[i],
        color="coral",
        **kwds,
    )
    ax.yaxis.set_major_locator(mticker.MultipleLocator(loc_maj[i]))
    ax.yaxis.set_minor_locator(mticker.MultipleLocator(loc_min[i]))

    ax.set_ylabel(labels[i])

# plot cwp sensitivity
ax_cwp.scatter(
    da_dist.sel(time=ds_anc.time.sel(time=ds_a.time)),
    sigmoid(ds_anc.dist_sic_0_50.sel(time=ds_a.time), *popt) * 1e3,
    color=cmc.batlow(0),
    label="CWP det.",
    **kwds,
)

# plot era5
ax_cwp.scatter(
    da_dist.sel(time=ds_anc.time.sel(time=ds_a.time)),
    ds_anc.era5_tclw.sel(time=ds_a.time) * 1e3,
    color=cmc.batlow(0.5),
    label="ERA5",
    **kwds,
)

# plot kt19
ax_tas.scatter(
    da_dist.sel(time=ds_anc.time.sel(time=ds_a.time)),
    ds_anc.kt19_bt.where(ds_anc.ix_clear_sky_kt19).sel(time=ds_a.time) / 0.995,
    color=cmc.batlow(0.25),
    label="KT-19",
    **kwds,
)

for ax in axes.flat:
    ax.xaxis.set_major_locator(mticker.MultipleLocator(100))
    ax.xaxis.set_minor_locator(mticker.MultipleLocator(25))
    ax.set_xlim(0, da_dist.max())

ax_cwp.set_ylim(cwp_ylim[version])
ax_wscl.set_ylim(0.05, 0.25)
ax_dhcl.set_ylim(0.1, 0.6)
ax_wsd.set_ylim(10, 30)
axes_y[0].set_ylim(bottom=210)
axes_y[-1].set_ylim(bottom=210)

ax_cwp.yaxis.set_major_locator(mticker.MultipleLocator(100))

handles = []
labels = []
for ax in [axes_y[0], ax_cwp, ax_tas]:
    h, l = ax.get_legend_handles_labels()
    handles += h
    labels += l
fig.legend(
    handles,
    labels,
    loc="lower center",
    frameon=True,
    markerscale=6,
    ncol=6,
    bbox_to_anchor=(0.5, 1.01),
)

axes[-1, 0].set_xlabel("Distance [km]")
axes[-1, 1].set_xlabel("Distance [km]")

write_figure(
    fig,
    f"paper/{figure}.png",
    dpi=300,
    bbox_inches="tight",
)

plt.show()